# Regressor
1. load datasets
1. sample from datasets
1. construct features
1. construct labels (load from logs)

In [ ]:
# load datasets:

import importlib
import random
import argparse
import configparser
import numpy as np
import networkx as nx
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_sparse
from torch import Tensor
from torch.nn import Linear
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torch.optim as optim

from torch_geometric.utils import negative_sampling, to_networkx
from typing import Union, Tuple
from torch_geometric.typing import OptPairTensor, Adj, OptTensor, Size
from torch_sparse import SparseTensor, matmul
from torch_geometric.nn.conv import MessagePassing

from ogb.linkproppred import PygLinkPropPredDataset, Evaluator


import networkx as nx
import seaborn as sns
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

import scipy
import math


from dataset_utils import node_feature_utils
from dataset_utils.node_feature_utils import *
import my_utils as utils
import sys,os
sys.path.append(os.getcwd())


importlib.reload(utils)

In [ ]:
# Load specific dataset:




from PrepareDatasets import DATASETS
import my_utils
import dataset_utils


print(DATASETS.keys())
"""
    'REDDIT-BINARY': RedditBinary,
    'REDDIT-MULTI-5K': Reddit5K,
    'COLLAB': Collab,
    'IMDB-BINARY': IMDBBinary,
    'IMDB-MULTI': IMDBMulti,
    'ENZYMES': Enzymes,
    'PROTEINS': Proteins,
    'NCI1': NCI1,
    'DD': DD,
    "MUTAG": Mutag,
    'CSL': CSL
"""

data_names = ['PROTEINS']
data_names = ['DD']
data_names = ['ENZYMES']
data_names = ['NCI1']
data_names = ['IMDB-MULTI']
data_names = ['REDDIT-BINARY']
data_names = ['CIFAR10']
data_names = ['ogbg_molhiv']


# NOTE:new kernel:
data_names = ['DD', 'PROTEINS', 'ENZYMES']

data_names = ['ogbg_moltox21','ogbg-molbace']

data_names = ['MUTAG']
data_names = []
datasets_obj = {}
for k, v in DATASETS.items():
    if k not in data_names:
        continue
    print('loaded dataset, name:', k)
    dat = v(use_node_attrs=True)
    datasets_obj[k] = dat
    # print(type(dat.dataset.get_data()))

In [ ]:
# extract features: mean, std of: avg, pooling, cc, tri_num, cycle4_num, ..., kernel features

import dataset_utils.node_feature_utils as nfu
from scipy.stats import pearsonr



def extract_features(adjs, labels):
    
    def get_mean_std_corr(features, labels):
        
        mean = np.array(np.mean(features))
        std = np.array(np.std(features))
        x = np.array(features).reshape(-1)
        y = np.array(labels)
        corr , _ = pearsonr(x, y)
        
        if np.isnan(corr):
            corr = np.array([0])
        
        if not isinstance(corr, np.ndarray):
            corr = np.array([corr])
            
        return np.array([mean.item(), std.item(), corr.item()])

    
    # F1: avgD:
    avg_d = [nfu.graph_avg_degree(adj=adj) for adj in adjs]
    f_avgD = get_mean_std_corr(avg_d, labels)
    # F2: avgCC:
    avg_cc = [nfu.node_cc_avg_feature(adj=adj) for adj in adjs]
    f_avgCC = get_mean_std_corr(avg_cc, labels)
    
    # F3: avgD/N:
    avg_DN = [nfu.graph_avgDN_feature(adj=adj) for adj in adjs]
    f_avgDN = get_mean_std_corr(avg_DN, labels)
    
    # F4: node num N:
    avg_N = [adj.shape[0] for adj in adjs]
    f_avgN = get_mean_std_corr(avg_N, labels)
    
    # F5: labels
    f_Y = get_mean_std_corr(labels, labels)[:2]
    
    # F6: cycles:
    avg_cyc = [nfu.graph_cycle_feature(adj=adj,k='4-5-6-7') for adj in adjs]
    f_cyc4 = get_mean_std_corr([c[0] for c in avg_cyc], labels)
    f_cyc5 = get_mean_std_corr([c[1] for c in avg_cyc], labels)
    f_cyc6 = get_mean_std_corr([c[2] for c in avg_cyc], labels)
    f_cyc7 = get_mean_std_corr([c[3] for c in avg_cyc], labels)
    
        
    feas = np.concatenate([f_avgD, f_avgCC, f_avgDN, f_avgN, f_Y, f_cyc4, f_cyc5, f_cyc6, f_cyc7], axis=0)
    return feas
    
    

In [ ]:
dataset = datasets_obj['MUTAG']
adjs = dataset.get_dense_adjs(dataset.dataset)
labels = [d.y for d in dataset.dataset]

feas = extract_features(adjs=adjs, labels=labels)

In [ ]:
# TODO: load each fold logs to construct y:
import json

# MUTAG:
# GIN_degree_log_path: [result_GIN_0403_GIN_degree_MUTAG]
# MLP_log_path: [result_GIN_0403_GIN_degree_MUTAG]
# GIN_degree_log_path: [result_GIN_0403_GIN_degree_MUTAG]
"""
{"best_config": {"config": {"model": "GIN", "device": "cuda:1", "batch_size": 64, "learning_rate": 0.0001, "classifier_epochs": 200, "hidden_units": [64, 64, 64, 64], "layer_num": 5, "optimizer": "Adam", "scheduler": {"class": "StepLR", "args": {"step_size": 50, "gamma": 0.5}}, "loss": "MulticlassClassificationLoss", "train_eps": false, "l2": 0.0, "aggregation": "sum", "gradient_clipping": null, "dropout": 0.5, "early_stopper": {"class": "Patience", "args": {"patience": 50, "use_loss": false}}, "shuffle": true, "resume": false, "additional_features": "degree", "node_attribute": false, "shuffle_feature": false, "roc_auc": false, "mol_split": false, "dataset": "syn_cc", "config_file": "gnn_comparison/config_GIN_lzd_degree.yml", "experiment": "endtoend", "result_folder": "results/result_0422_GIN_lzd_degree_syn_cc_0.1", "dataset_name": "syn_cc", "dataset_para": "0.1", "outer_folds": 10, "outer_processes": 2, "inner_folds": 5, "inner_processes": 1, "debug": true, "ogb_evl": false}, "TR_score": 16.183574925298277, "VL_score": 21.505376272304083, "TR_roc_auc": -1, "VL_roc_auc": -1}, "OUTER_TR": 14.774557204254199, "OUTER_TS": 11.003236511378612, "OUTER_TR_ROCAUC": -1, "OUTER_TE_ROCAUC": -1}
"""

_OUTER_RESULTS_FILENAME = 'outer_results.json'

def get_test_acc(data_root_path, fold=10):
    if data_root_path is None:
        return [None for _ in range(fold)]
    
    outer_TR_scores,outer_TS_scores,outer_TR_ROCAUC,outer_TE_ROCAUC = [],[],[],[]
    for i in range(1, fold+1):
        config_filename = os.path.join(data_root_path, f'OUTER_FOLD_{i}', _OUTER_RESULTS_FILENAME)

        with open(config_filename, 'r') as fp:
            outer_fold_scores = json.load(fp)

            outer_TR_scores.append(outer_fold_scores['OUTER_TR'])
            outer_TS_scores.append(outer_fold_scores['OUTER_TS'])
            
            if 'OUTER_TR_ROCAUC' in outer_fold_scores:
                outer_TR_ROCAUC.append(outer_fold_scores['OUTER_TR_ROCAUC'])
                outer_TE_ROCAUC.append(outer_fold_scores['OUTER_TE_ROCAUC'])

    return outer_TS_scores


# Load splits and logs for each fold

In [ ]:
# syn_cc datasets:

syn_cc_GNN_test_acc = []

for i in range(1, 10):
    data_root_path = f'./results/result_0422_GIN_lzd_degree_syn_cc_{i/10}/GIN_syn_cc_assessment/10_NESTED_CV'
    syn_cc_GNN_test_acc.append(get_test_acc(data_root_path))
    

syn_cc_MLP_test_acc = []

for i in range(1, 10):
    data_root_path = f'./results/result_0422_Baseline_lzd_mlp_syn_cc_{i/10}/MolecularGraphMLP_syn_cc_assessment/10_NESTED_CV'
    syn_cc_MLP_test_acc.append(get_test_acc(data_root_path))  

In [ ]:
# construct E of each fold, and plot

# Effectiveness 

def get_E(Acc_MLP_avg_degree, Acc_GNN_degree, Acc_MLP_attr, Acc_GNN_attr):
    factor = 0.5
    if Acc_MLP_avg_degree is None:
        E_struct = 0
        factor = 1
    else:
        E_struct = (abs(Acc_GNN_degree - Acc_MLP_avg_degree) / Acc_MLP_avg_degree) * (100 - min(Acc_GNN_degree, Acc_MLP_avg_degree))
    
    if Acc_MLP_attr is None:
        E_attribute = 0
        factor = 1
    else:
        E_attribute = (abs(Acc_GNN_attr - Acc_MLP_attr) / Acc_MLP_attr) * (100 - min(Acc_GNN_attr, Acc_MLP_attr))
    return (E_struct+E_attribute) * factor

# NOTE: get E for each dataset:

def plot_E(es, ax=None):
    e_res = sorted(es, key=lambda x:x[0])
    labels = [e[1] for e in e_res]

    if ax is None:
        fig, ax = plt.subplots(dpi=100)
        
    for e in e_res:
        bars = ax.bar(e[1], e[0], label=e[1], hatch='\\', edgecolor='black', linewidth=0.5)

    ax.set_xticks(range(len(labels)))
    ax.set_xticklabels(labels, rotation=45, ha='center')
    ax.set_axisbelow(True)
    ax.grid(linestyle='dashed',zorder=0)
    ax.set_title('E=(E_struct+E_attr)/2')

# ax.legend()

In [ ]:
sy_es = []
for i in range(len(syn_cc_GNN_test_acc)):
    # cc_MLP_avg_degree, Acc_GNN_degree, Acc_MLP_attr, Acc_GNN_att
    sy_es.append((get_E(np.mean(syn_cc_MLP_test_acc[i]), np.mean(syn_cc_GNN_test_acc[i]),
                       None, None), f'cc_corr:{(i+1)/10}'))
# 
plot_E(sy_es)

In [ ]:
 # Check mutag
# construct each E with each fold and

data_root_path = f'./results/result_GIN_0404_GIN_attr_MUTAG/GIN_MUTAG_assessment/10_NESTED_CV'
mutag_GNN_attr_test_acc=get_test_acc(data_root_path)


data_root_path = f'./results/result_GIN_0327_finger_mlp_attr_multicrossen_MUTAG/MolecularFingerprint_MUTAG_assessment/10_NESTED_CV'
mutag_MLP_attr_test_acc = get_test_acc(data_root_path)

In [ ]:
# Check mutag
# construct each E with each fold and

data_root_path = f'./results/result_GIN_0404_GIN_attr_MUTAG/GIN_MUTAG_assessment/10_NESTED_CV'
mutag_GNN_attr_test_acc=get_test_acc(data_root_path)


data_root_path = f'./results/result_GIN_0327_finger_mlp_attr_multicrossen_MUTAG/MolecularFingerprint_MUTAG_assessment/10_NESTED_CV'
mutag_MLP_attr_test_acc = get_test_acc(data_root_path)



mutag_es = []
for i in range(len(mutag_GNN_attr_test_acc)):
    # cc_MLP_avg_degree, Acc_GNN_degree, Acc_MLP_attr, Acc_GNN_att
    mutag_es.append((get_E(None, None,
                       mutag_MLP_attr_test_acc[i], mutag_GNN_attr_test_acc[i]), f'mutag_fold:{(i+1)/10}'))

# plot_E(mutag_es)

In [ ]:
# load splits as datasets:
import torch_geometric.utils as torch_utils
    
"""
the ixj th sample : (D_i_split_j, E_i_j)
"""

# Check mutag
# construct each E with each fold and
def is_pyg_dataset(d_name:str):
    return d_name.startswith('ogb') or d_name.startswith('syn')

def E_datasets(dataset, 
               MLP_log_path_struct=None, GNN_log_path_struct=None,
               MLP_log_path_attr=None, GNN_log_path_attr=None, fold=10):
    
    
    GNN_test_acc_struct = get_test_acc(GNN_log_path_struct, fold=fold)
    MLP_test_acc_struct = get_test_acc(MLP_log_path_struct, fold=fold)
    GNN_test_acc_attr = get_test_acc(GNN_log_path_attr, fold=fold)
    MLP_test_acc_attr = get_test_acc(MLP_log_path_attr, fold=fold)
    
    def get_dense_adjs(dataset):
        adjs = []

        if is_pyg_dataset(dataset.name):
            for d in dataset:
                if d.edge_index.numel() < 1:
                    N = d.x.shape[0]
                    adj = np.ones(shape=(N, N))
                else:
                    adj = torch_utils.to_dense_adj(d.edge_index).numpy()[0]
                adjs.append(adj)
        else:
            adjs = [d.to_numpy_array() for d in dataset.data]
            
        return adjs
    
    mutag_splits = []
    for i in range(fold):
        train_loader, val_loader = dataset.get_model_selection_fold(outer_idx=i, inner_idx=0, batch_size=1, shuffle=False)
        adjs = get_dense_adjs(train_loader.dataset) + get_dense_adjs(val_loader.dataset)
        labels = [d.y for d in train_loader.dataset] + [d.y for d in val_loader.dataset]
        feas = extract_features(adjs=adjs, labels=labels)
        e = get_E(MLP_test_acc_struct[i], GNN_test_acc_struct[i],  MLP_test_acc_attr[i], GNN_test_acc_attr[i])
            
        mutag_splits.append((feas, e))
        
    return mutag_splits
        
        

In [ ]:
# MUTAG
GNN_attr_log_path = f'./results/result_GIN_0404_GIN_attr_MUTAG/GIN_MUTAG_assessment/10_NESTED_CV'
MLP_attr_log_path = f'./results/result_GIN_0327_finger_mlp_attr_multicrossen_MUTAG/MolecularFingerprint_MUTAG_assessment/10_NESTED_CV'
dataset = datasets_obj['MUTAG']
mutag_datasets = E_datasets(dataset, MLP_attr_log_path, GNN_attr_log_path, None, None)


In [ ]:
del datasets_obj

# DD:

data_names = ['DD']
              
datasets_obj = {}
for k, v in DATASETS.items():
    if k not in data_names:
        continue
    print('loaded dataset, name:', k)
    dat = v(use_node_attrs=True)
    datasets_obj[k] = dat
    # print(type(dat.dataset.get_data()))

In [ ]:
# DD
"""
result_GIN_0308_DD_degree_attribute
result_GIN_0309_DD_both_degree_attribute
result_GIN_0310_DD_mix
result_GIN_0311_DD_mix
result_GIN_0312_DD_mix
result_GIN_0318_decouple_attr_degree_DD
result_GIN_0319_new_alpha_decouple_attr_degree_DD
result_GIN_0327_finger_mlp_attr_crossen_DD
result_GIN_0327_finger_mlp_attr_multicrossen_DD
"""

GNN_attr_log_path = f'./results/result_GIN_0308_DD_degree_attribute/Adapter_DD_assessment/10_NESTED_CV'
MLP_attr_log_path = f'./results/result_GIN_0327_finger_mlp_attr_multicrossen_DD/MolecularFingerprint_DD_assessment/10_NESTED_CV'
dataset = datasets_obj['DD']
dd_datasets = E_datasets(dataset, None, None, GNN_attr_log_path, MLP_attr_log_path)



In [ ]:
# save datasets
import pickle as pk

def save_datasets(datasets, file_name):
    with open(file_name, 'wb') as f:
        pk.dump(datasets, f)


# save_datasets(mutag_datasets, 'mutag_datasets.pkl')
# save_datasets(dd_datasets, 'dd_datasets.pkl')




In [ ]:
del datasets_obj

# DD:

data_names = ['CIFAR10']
              
datasets_obj = {}
for k, v in DATASETS.items():
    if k not in data_names:
        continue
    print('loaded dataset, name:', k)
    dat = v(use_node_attrs=True)
    datasets_obj[k] = dat
    # print(type(dat.dataset.get_data()))

In [ ]:
# CIFAR10 only has on fold
"""
result_GIN_0317_decouple_degree_attr_CIFAR10
result_GIN_0317_mix_degree_attr_CIFAR10
result_GIN_0317_only_attr_CIFAR10
result_GIN_0317_only_degree_CIFAR10
result_GIN_0318_decouple_degree_attr_CIFAR10
result_GIN_0327_finger_mlp_attr_multicrossen_CIFAR10
result_GIN_0401_GIN_degree_CIFAR10
result_GIN_0403_GIN_degree_CIFAR10
"""

MLP_log_path_degree = f'./results/result_GIN_0401_graph_mlp_avgDegree_CIFAR10/MolecularGraphMLP_CIFAR10_assessment/1_NESTED_CV'
GNN_log_path_degree = f'./results/result_GIN_0317_only_degree_CIFAR10/GIN_CIFAR10_assessment/1_NESTED_CV'

MLP_log_path_attr = f'./results/result_GIN_0327_finger_mlp_attr_multicrossen_CIFAR10/MolecularFingerprint_CIFAR10_assessment/10_NESTED_CV'
GNN_log_path_attr = f'./results/result_GIN_0317_only_attr_CIFAR10/GIN_CIFAR10_assessment/1_NESTED_CV'

dataset = datasets_obj['CIFAR10']
cifar10_datasets = E_datasets(dataset, MLP_log_path_degree, GNN_log_path_degree, MLP_log_path_attr, GNN_log_path_attr, fold=1)



In [ ]:
save_datasets(dd_datasets, 'cifar10_datasets.pkl')

del datasets_obj


In [ ]:
# get NCI1

# 'NCI1', 'IMDB-MULTI', 'REDDIT-BINARY', 'CIFAR10',
# #               'ogbg_molhiv', 'ogbg_moltox21', 'ogbg-molbace', 'MUTAG']

"""
result_GIN_0312_attr_degree_NCI1
result_GIN_0312_decouple_attr_degree_NCI1
result_GIN_0312_only_attribute_NCI
result_GIN_0312_only_degree_NCI
result_GIN_0313_degree_shuffle_NCI1
result_GIN_0313_pagerank_NCI1
result_GIN_0318_decouple_attr_degree_NCI1
result_GIN_0319_new_alpha_decouple_attr_degree_NCI1
result_GIN_0327_finger_mlp_attr_crossen_NCI1
result_GIN_0327_finger_mlp_attr_multicrossen_NCI1
result_GIN_0403_GIN_attr_NCI1
result_GIN_0403_GIN_degree_NCI1
result_GIN_0404_GIN_attr_NCI1
"""
del datasets_obj

data_names = ['NCI1']
              
datasets_obj = {}
for k, v in DATASETS.items():
    if k not in data_names:
        continue
    print('loaded dataset, name:', k)
    dat = v(use_node_attrs=True)
    datasets_obj[k] = dat

MLP_log_path_degree = f'./results/result_GIN_0401_graph_mlp_avgDegree_CIFAR10/MolecularGraphMLP_CIFAR10_assessment/1_NESTED_CV'
GNN_log_path_degree = f'./results/result_GIN_0317_only_degree_CIFAR10/GIN_CIFAR10_assessment/1_NESTED_CV'

MLP_log_path_attr = f'./results/result_GIN_0327_finger_mlp_attr_multicrossen_NCI1/MolecularFingerprint_NCI1_assessment/10_NESTED_CV'
GNN_log_path_attr = f'./results/result_GIN_0404_GIN_attr_NCI1/GIN_NCI1_assessment/1_NESTED_CV'

dataset = datasets_obj['NCI1']
cifar10_datasets = E_datasets(dataset, MLP_log_path_degree, GNN_log_path_degree, MLP_log_path_attr, GNN_log_path_attr, fold=1)




# Load regressor datasets

In [ ]:
# load all pickle datasets
import pickle as pk

def load_datasets(file_name):
    with open(file_name, 'rb') as f:
        datasets = pk.load(f)
    return datasets

d1 = load_datasets('mutag_datasets.pkl')
d2 = load_datasets('dd_datasets.pkl')
d3 = load_datasets('cifar10_datasets.pkl')



In [ ]:
# SynCC

data_names = ['syn_cc']
              
syn_datasets = []
for i in range(1, 10):
    v = DATASETS['syn_cc']
    config = {'dataset_para':f'{i/10}'}
    dat = v(use_node_attrs=True, config=config)

    gnn_data_root_path = f'./results/result_0422_GIN_lzd_degree_syn_cc_{i/10}/GIN_syn_cc_assessment/10_NESTED_CV'
    mlp_data_root_path = f'./results/result_0422_Baseline_lzd_mlp_syn_cc_{i/10}/MolecularGraphMLP_syn_cc_assessment/10_NESTED_CV'
    e_dataset_10_folds = E_datasets(dat, mlp_data_root_path, gnn_data_root_path, None, None)
    syn_datasets.extend(e_dataset_10_folds)


In [ ]:
save_datasets(syn_datasets, 'syn_datasets.pkl')

In [ ]:
syn_datasets = load_datasets('syn_datasets.pkl')


In [ ]:
# combine all datasets:


all_datasets = []


for d in syn_datasets:
    all_datasets.append(d)
    
all_datasets.extend(d1+d2+d3)

store_each_len = []

for d in all_datasets:
    store_each_len.append(len(d))
    
def belong_to_which_datasets(idx, store_each_len):
    sum_len = 0
    for i, l in enumerate(store_each_len):
        sum_len += l
        if idx < sum_len:
            return i
    return -1

# construct regressor

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
import matplotlib.pyplot as plt


import numpy as np
from sklearn.preprocessing import StandardScaler


def mean_norm(x):
    scaler = StandardScaler()
    return scaler.fit_transform(x), scaler


class CustomDataset(Dataset):
    def __init__(self, data):
        self.X = torch.tensor([t[0] for t in data], dtype=torch.float32)
        self.Y = torch.tensor([t[1] for t in data], dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]


np.random.seed(42)

def generate_data(num_samples):
    features = np.random.rand(num_samples, 26)
    labels = np.random.rand(num_samples, 1)
    return [(features[i], labels[i]) for i in range(num_samples)]



X = np.array([t[0] for t in all_datasets])
Y = np.array([t[1] for t in all_datasets])

normalized_x, scaler_x = mean_norm(X)
normalized_y, scaler_y = mean_norm(Y.reshape(-1, 1))

print(normalized_x.shape, normalized_y.shape)

print(len(normalized_x))
normed_combined_data = [(normalized_x[i], normalized_y[i]) for i in range(normalized_x.shape[0])]

dataset = CustomDataset(normed_combined_data)

train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

class MLPRegressor(nn.Module):
    def __init__(self):
        super(MLPRegressor, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(26, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.Dropout(0.4),
            nn.Sigmoid(),
            nn.Linear(64, 1),
        )

    def forward(self, x):
        return self.layers(x)


    
    
def evaluate(loader, model,):
    model.eval()
    total_loss = 0.0
    total_samples = 0
    preds = []
    Y = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels.view(-1, 1))
            total_loss += loss.item() * inputs.size(0)
            total_samples += inputs.size(0)
            preds.append(outputs.cpu().numpy())
            Y.append(labels.cpu().numpy())
            
    
    preds = np.concatenate(preds, axis=0)
    Y = np.concatenate(Y, axis=0).ravel()
    
    mae = mean_absolute_error(Y, preds)
    mse = mean_squared_error(Y, preds)
    rmse = math.sqrt(mse)
    r2 = r2_score(Y, preds)
    
    results = (mae, mse, rmse, r2)
    return mse, results, preds, Y

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



model = MLPRegressor().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 500
train_losses, val_losses = [], []

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.view(-1, 1))
        loss.backward()
        optimizer.step()

    train_loss, _, _, _ = evaluate(train_loader, model)
    val_loss, _,_,_ = evaluate(val_loader, model)
    train_losses.append(train_loss)
    val_losses.append(val_loss)

    # print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

test_loss, test_results,_, _ = evaluate(test_loader, model)
print(f"Test Loss: {test_loss:.4f}")
print(f'MLPs: {round(test_results[0], 2)} & {round(test_results[1], 2)} & {round(test_results[2], 2)} & {round(test_results[3], 2)}')

# Plot train and validation loss curves
plt.figure()
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.legend()
plt.title("Train and Validation Loss Curves")
plt.show()


In [ ]:

np.random.seed(42)

def train_regressor(loader, regressor):
    X = []
    Y = []

    for inputs, labels in loader:
        X.extend(inputs.cpu().numpy())
        Y.extend(labels.cpu().numpy())

    X = np.array(X)
    Y = np.array(Y).ravel()

    regressor.fit(X, Y)
    

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import math


def regressor_evaluate(loader, regressor):
    X = []
    Y = []

    for inputs, labels in loader:
        X.extend(inputs.cpu().numpy())
        Y.extend(labels.cpu().numpy())

    X = np.array(X)
    Y = np.array(Y).ravel()

    preds = regressor.predict(X)
    mae = mean_absolute_error(Y, preds)
    mse = mean_squared_error(Y, preds)
    rmse = math.sqrt(mse)
    r2 = r2_score(Y, preds)
    
    results = (mae, mse, rmse, r2)
    return mse, results, preds, Y


# baseline regressors:
- random forest

In [ ]:

def run_regressor(regressor, regressor_name):
        
    # Choose a regressor
    # regressor = GradientBoostingRegressor(random_state=42)

    train_regressor(train_loader, regressor)

    train_loss, train_results, train_preds, train_Y = regressor_evaluate(train_loader, regressor)
    val_loss, val_results, val_preds, val_Y = regressor_evaluate(val_loader, regressor)
    test_loss, test_reuslts, test_preds, test_Y = regressor_evaluate(test_loader, regressor)

    # print(f"regressor_name: {regressor_name}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Test Loss: {test_loss:.4f} \
    #       (mse, rmse, r^2): {test_reuslts}")
    
    print(f'{regressor_name}: {round(test_reuslts[0], 2)} & {round(test_reuslts[1], 2)} & {round(test_reuslts[2], 2)} & {round(test_reuslts[3], 2)}')
    plt.figure()
    plt.plot(scaler_y.inverse_transform(test_preds), label="Predictions")
    plt.plot(scaler_y.inverse_transform(test_Y), label="True Labels")
    plt.title(regressor_name)
    plt.legend()


from sklearn.svm import SVR
from sklearn.linear_model import Ridge

# Choose a regressor
regressors = {'SVR': SVR(),'Ridge': Ridge(), "RandomForestRegressor": RandomForestRegressor(),  
              "LinearRegression": LinearRegression()}

for k, regressor in regressors.items():
    run_regressor(regressor, k)

In [ ]:
import xgboost as xgb
import lightgbm as lgb


regressors = {'XG boost': xgb.XGBRegressor(objective ='reg:squarederror'),
              'LGBMRegressor': lgb.LGBMRegressor()}

for k, regressor in regressors.items():
    run_regressor(regressor, k)

In [ ]:
# use test dataset to evaluate the model

# average each datasets:

from collections import Counter

test_dataset_idx = [belong_to_which_datasets(i, store_each_len) for i in test_data.indices]
print(Counter(test_dataset_idx))

# Get predictions and true labels from the test dataset
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predictions.extend(outputs.squeeze().cpu().numpy())
        true_labels.extend(labels.squeeze().cpu().numpy())


plt.figure()
plt.plot(scaler_y.inverse_transform(predictions), label="Predictions")
plt.plot(scaler_y.inverse_transform(true_labels), label="True Labels")
plt.legend()


# # Inverse normalization of true_labels and predictions
# combined_labels = np.array([t[1] for t in combined_data])
# label_norm = np.linalg.norm(combined_labels)
# inverse_true_labels = np.multiply(true_labels, label_norm)
# inverse_predictions = np.multiply(predictions, label_norm)

# # Plot predicted vs true labels
# plt.figure()
# plt.scatter(true_labels, inverse_predictions, alpha=0.5)
# plt.xlabel("True Labels")
# plt.ylabel("Predicted Labels")
# plt.title("Predicted vs True Labels")
# plt.show()
